In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd
import numpy as np

In [2]:
model=load_model('model.h5')

with open('onehot_geo.pkl','rb') as file:
    onehot_geo=pickle.load(file)

with open('label_gender.pkl','rb') as file:
    label_gender=pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)



In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [6]:
# One-hot encode 'Geography'
geo_encoded = onehot_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_geo.get_feature_names_out(['Geography']))
geo_encoded_df


C:\Users\kk061\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [9]:
## Encode categorical variables
input_df['Gender']=label_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [10]:
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [11]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.54007722,  0.91186722,  0.10685843, -0.69164931, -0.26034447,
         0.81311138,  0.64946204,  0.96978946, -0.88017893,  1.00053348,
        -0.57776083, -0.57735027]])

In [12]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


array([[0.04150536]], dtype=float32)

In [13]:
prediction_proba = prediction[0][0]
prediction_proba

0.041505363

In [14]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
